In [1]:
def sir_11(X, y, num_slices, K):
    X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    # Step 4: Center the predictor means
    X_centered = (X_means - np.mean(X, axis=0))
    # X_centered = (X_means - np.mean(X, axis=0))/np.linalg.norm(X_means - np.mean(X, axis=0))
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    return edr_est, V_hat

In [2]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
time = 30
def ave(arr, N):
    for i in range(len(arr)):
        arr[i] = arr[i]/N 
    return arr 
def compute_eigen(Q4, P, K):
    eigenvalues2, eigenvectors2 = np.linalg.eig(Q4)
    K_index = np.argpartition(np.abs(eigenvalues2), P - K) >= P - K
    K_largest_eigenvectors = eigenvectors2[:, K_index]
    edr_est = K_largest_eigenvectors  
    if edr_est[0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    return edr_est    
def proj(edr_est): 
    E = edr_est @ np.linalg.inv(edr_est.T @ edr_est) @ edr_est.T
    return E
def exhi(obj1):        
    array1 = np.vectorize(lambda x: f"{x:.6f}")(obj1)
    table = tabulate(array1, tablefmt='latex_raw')
    lines = table.split('\n')
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    print(latex_table)    
def MSE(X, y):
    Mse = 0
    # Split using time series cross-validation (e.g., 5 splits)
    train_window = int(np.round(0.75 * len(X), 0))  # Fixed size of training set (e.g., 75 time steps)
    test_window = 1   # Fixed size of test set (e.g., 25 time steps)
    # Sliding window cross-validation
    for i in range(0, len(X) - train_window - test_window + 1):
        X_train = X[i:i + train_window]
        y_train = y[i:i + train_window]
        X_test = X[i + train_window:i + train_window + test_window]
        y_test = y[i + train_window:i + train_window + test_window]
        model = LinearRegression()
        model.fit(X_train, y_train)
        # Make predictions
        y_pred = model.predict(X_test)
        # Calculate errors
        mse = mean_squared_error(y_test, y_pred)
        Mse += mse 
    Mse = Mse/(len(X) - train_window - test_window)
    return np.sqrt(Mse)
def optimal_values(arr):
    if len(arr) == 0:
        return None  # Handle the case of an empty array
    smallest = arr[0]  # Assume the first element is the smallest
    for num in arr:
        if num < smallest:
            smallest = num
    return smallest   
def optimal_Q(arr):
    if len(arr) == 0:
        return None  # Handle the case of an empty array
    smallest = arr[0]  # Assume the first element is the smallest
    for num in arr:
        if num < smallest:
            smallest = num
    for i in range(len(arr)):
        if arr[i] == smallest:
            return i # Return the first matched index
    return -1
def plotyy(x, li1, li2, li3, label1 ,label2 ,label3, title, xlabel, ylabel):
    # plt.plot(x, obj1_optimal_values, label= label_variable, color='blue', marker='o')
    plt.plot(x, li1, label = label1, color='blue', marker='o', linestyle='--', alpha = 0.5)
    plt.plot(x, li2, label = label2, color='red', marker='^', linestyle='-.', alpha = 0.7)
    plt.plot(x, li3, label = label3, color='black', marker='s', alpha = 0.3)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='upper left')
    plt.tight_layout()
def Generation(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, sigma, n_rep):
    p = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; n_burn_round = 2; P1 = p*P; AR = initial_value 
    noise1 = np.zeros((num_N, n_burn))
    noise2 = np.zeros((num_N, (n_rep+1)*n_obs+time*n_rep+n_rep*S))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    # error_1_1 is for dim 2P, error_1_2 is for dim P
    error_1_1 = [np.zeros((1, )) for _ in range(S)]
    error_1_2 = [np.zeros((1, )) for _ in range(S)]
    error_2 = [np.zeros((1, )) for _ in range(S)]
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    # True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    for h in range(num_N):
        noise1[h] = np.random.normal(0, 1, size=(n_burn)) 
        noise2[h] = np.random.normal(0, 1, size=((n_rep+1)*n_obs+time*n_rep+n_rep*S))
    ar_series = np.zeros((num_N, (n_rep+1)*n_obs+time*n_rep+n_rep*S))
    for t in range(1, n_burn + 1):
        for i in range(4):
            AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
            AR[i][1] = AR[i][0]
            AR[i][0] = AR_temp
        for i in range(4):
            for k in range(n_burn_round):
                ar_series[i][k] = AR[i][k]
        for t in range(2, n_obs+S+3):
            for i in range(4):
                ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
        for a in range(2, S+2):
            y[a-2] = ar_series[4][a:n_obs+a]
            X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T 
        X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.zeros((P, n_obs))), axis = 0).T
        X3 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
        phi = ar_coeff[0] + ar_coeff[1]
        X = X1[0]
        V1 = []
        for a in range(0, S):
            if a == 0:
                _, M = sir_11(X2, y[a], H, K)
                V1.append(M)
            else:    
                _, M = sir_11(X, y[a], H, K)
                V1.append(M)
    # replacing the replication step
    n = 0
    t_start = 0
    t_end = n_obs + S + 1
    # enlarge 10???
    while n < n_rep:  
        # pause every 10 time points and use the next (number of samples = n_obs) time points
        df = pd.DataFrame(ar_series[:, t_start:  t_end])
        df.to_excel(fr'C:\Users\yhuang73\Desktop\datatestAR2_{n_obs}_{sigma}_{n_rep}\array_data_numberofsamples{n_obs}_replicas{n}_AR2_{"_".join(map(str, ar_coeff))}.xlsx', index=False, header=False)
        t_start += n_obs + time 
        t_end += n_obs + time
        # if t_end > len(ar_series):
        #     t_end = len(ar_series)
        #     break
        n += 1
        
def Calculation(ar_series_xlsx, ar_coeff, H, S, n):
    alpha = 1e-5
    p = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; n_burn_round = 2; P1 = p*P; AR = initial_value 
    noise1 = np.zeros((num_N, n_burn))
    noise2 = np.zeros((num_N, (n_rep+1)*n_obs+time*n_rep+n_rep*S))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    prediction_mse_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    # error_1_1 is for dim 2P, error_1_2 is for dim P
    error_1 = [np.zeros((1, )) for _ in range(S)]
    error_2 = [np.zeros((1, )) for _ in range(S)]
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    # True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))
    # error_1_3 = 0
    # prediction_mse_1_3 = 0
    # proj_error_1_3 = 0
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    df = pd.read_excel(ar_series_xlsx, header=None)
    ar_series = df.to_numpy()
    for a in range(2, S+2):
        y[a-2] = ar_series[4][a:n_obs+a]
        X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T 
    X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.zeros((P, n_obs))), axis = 0).T
    X3 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
    phi = ar_coeff[0] + ar_coeff[1]
    X = X1[0]
    V1 = []
    for a in range(0, S):
        _, M = sir_11(X, y[a], H, K)
        V1.append(M)
    # objective 1: experiment
    zero_1 = np.zeros((P, (p-1)*P))  # Shape (P, (p-1)*P)
    zero_2 = np.zeros(((p-1)*P, P))  # Shape ((p-1)*P, P)
    zero_3 = np.zeros(((p-1)*P, (p-1)*P))  # Shape ((p-1)*P, (p-1)*P)
    # Assemble the block matrix
    top_block = np.hstack([np.linalg.inv(np.cov(X3.T) + alpha * np.eye((X3.T).shape[0])), zero_1])  # Top block
    bottom_block = np.hstack([zero_2, zero_3])  # Bottom block
    Mhat = np.vstack([top_block, bottom_block]) 
    for q in range(0, S):
        Q3 = np.zeros((P1, P1))
        for j in range(P1):
            for k in range(P1):
                # np.linalg.inv(np.cov(X2.T)) is invalid, have to use np.linalg.pinv(np.cov(X2.T))??
                Q3[j, k] = (Mhat @ V1[0] @ Mhat)[j, k] + sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T) + alpha * np.eye((X1[q].T).shape[0])) @ V1[a] @ (np.linalg.inv(np.cov(X.T) + alpha * np.eye((X1[q].T).shape[0]))))[j, k] * (phi[k] ** a) for a in range(1, q + 1))

        edr_est = compute_eigen(Q3, P1, K)
        for i in range(1, p):
            for j in range(P):
                edr_est[i*P + j] = edr_est[i*P + j]/ar_coeff[i-1][j]
        edr_est = np.real(edr_est / np.linalg.norm(edr_est))
        # additional step, dimension P, average every P dimensional vector. 
        average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
        error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
        prediction_mse_1[q] += MSE((X3 @ average_edr_est).reshape(-1,1), y[0])
        proj_error_1[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
    # objective 2: experiment
    for q in range(0, S):
        if q == 0:
            Q4 = Mhat @ V1[0] @ Mhat
        else:
            #np.linalg.inv(np.cov(X1[q].T) + )
            Q4 = np.linalg.inv(np.cov(X1[q].T) + alpha * np.eye((X1[q].T).shape[0])) @ V1[q] @ np.linalg.inv(np.cov(X1[q].T) + alpha * np.eye((X1[q].T).shape[0]))   # multiply np.linalg.inv(np.cov(X1[q].T)), by stationarity, it should cause no influence.
        # Q3 = np.linalg.inv(np.cov(X.T)) @ V1[q] @ np.linalg.inv(np.cov(X.T)), if we multiply np.linalg.inv(np.cov(X.T)) like this line, result for q = 0 should be the same.   
        K_largest_eigenvectors = compute_eigen(Q4, P1, K)
        edr_est1 = np.multiply(np.power(ar_coeff, -q).reshape(-1,1), K_largest_eigenvectors.flatten())
        for i in range(1, p):
            for j in range(P):
                edr_est1[i*P + j] = edr_est1[i*P + j]/ar_coeff[i-1][j]
        edr_est1 = np.real(edr_est1 / np.linalg.norm(edr_est1))
        average_edr_est1 = np.mean(edr_est1.reshape(-1, P), axis=0)
        error_2[q] += abs(average_edr_est1[0] / average_edr_est1[1] - a1/a2)
        prediction_mse_2[q] += MSE((X3 @ average_edr_est1).reshape(-1,1), y[0])
        proj_error_2[q] += np.linalg.norm((proj((average_edr_est1).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
    return error_1, prediction_mse_1, proj_error_1, error_2, prediction_mse_2, proj_error_2  
def Evaluation(error_1, prediction_mse_1, proj_error_1, error_2, prediction_mse_2, proj_error_2, H, S, n_rep):
    num_N = 5;P = 4;K = 1;l = 1
    # Exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    # Exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    # exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    # exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    error_1 = ave(error_1, n_rep)    
    error_2 = ave(error_2, n_rep)
    prediction_mse_1 = ave(prediction_mse_1, n_rep)
    prediction_mse_2 = ave(prediction_mse_2, n_rep)
    proj_error_1 = ave(proj_error_1, n_rep)
    proj_error_2 = ave(proj_error_2, n_rep)
    index_array = list(range(S))
    SIR_values = [error_1[0], prediction_mse_1[0], proj_error_1[0]]
    Obj1_optimal_values = [optimal_values(error_1), optimal_values(prediction_mse_1), optimal_values(proj_error_1)]
    Obj2_optimal_values = [optimal_values(error_2), optimal_values(prediction_mse_2), optimal_values(proj_error_2)]
    Obj1_optimal_Q = [optimal_Q(error_1),optimal_Q(prediction_mse_1),optimal_Q(proj_error_1)]
    Obj2_optimal_Q = [optimal_Q(error_2),optimal_Q(prediction_mse_2),optimal_Q(proj_error_2)]
    # for i in range(S):
    #     exhi_1[i] = error_1[i]
    #     exhi_1[i] = np.vstack((exhi_1[i], proj_error_1[i]))
    #     exhi_1[i] = np.vstack((exhi_1[i], prediction_mse_1[i]))
    #     exhi_1[i] = np.vstack((np.array([[index_array[i]]]), exhi_1[i]))
    # for i in range(S):
    #     exhi_2[i] = error_2[i]
    #     exhi_2[i] = np.vstack((exhi_2[i], proj_error_2[i]))
    #     exhi_2[i] = np.vstack((exhi_2[i], prediction_mse_2[i]))
    #     exhi_2[i] = np.vstack((np.array([[index_array[i]]]), exhi_2[i]))
    return SIR_values, Obj1_optimal_values, Obj2_optimal_values, Obj1_optimal_Q, Obj2_optimal_Q

def Graph(H, S, n_rep):
    for k in [0.2, 0.4, 0.6]:
        for i in np.arange(0.1, 1.0, 0.1):
            sir_values = [[], [], []]
            obj1_optimal_values = [[], [], []]
            obj2_optimal_values = [[], [], []]
            obj1_optimal_Q = [[],[],[]]
            obj2_optimal_Q = [[],[],[]]
            for j in np.arange(0.1, 1.0, 0.1):   
                Prediction_mse_1 = [np.zeros((1, 1)) for _ in range(S)]
                Prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
                Proj_error_1 = [np.zeros((1, )) for _ in range(S)] 
                Proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
                Error_1 = [np.zeros((1, )) for _ in range(S)]
                Error_2 = [np.zeros((1, )) for _ in range(S)]
                ar_coeff = [[round(val, 1) for val in [j, i, k, k]], [round(val, 2) for val in [0.1, 0.1, 0.1, 0.1]]]
                for n in range(n_rep):
                    filepath = fr'C:\Users\yhuang73\Desktop\datatestAR2_{n_obs}_{sigma}_{n_rep}\array_data_numberofsamples{n_obs}_replicas{n}_AR2_{"_".join(map(str, ar_coeff))}.xlsx'
                    error_1, prediction_mse_1, proj_error_1, error_2, prediction_mse_2, proj_error_2 = Calculation(filepath, ar_coeff, H, S, n)
                    for s in range(S):
                        Error_1[s] += error_1[s]
                        Prediction_mse_1[s] += prediction_mse_1[s]
                        Proj_error_1[s] += proj_error_1[s]
                        Error_2[s] += error_2[s]
                        Prediction_mse_2[s] += prediction_mse_2[s]
                        Proj_error_2[s] += proj_error_2[s]
                SIR_values, Obj1_optimal_values, Obj2_optimal_values, Obj1_optimal_Q, Obj2_optimal_Q = Evaluation(Error_1, Prediction_mse_1, Proj_error_1, Error_2, Prediction_mse_2, Proj_error_2, H, S, n_rep)
                
                for j in range(3):
                    sir_values[j].append(SIR_values[j])
                    obj1_optimal_values[j].append(Obj1_optimal_values[j])
                    obj2_optimal_values[j].append(Obj2_optimal_values[j])
                    obj1_optimal_Q[j].append(Obj1_optimal_Q[j])
                    obj2_optimal_Q[j].append(Obj2_optimal_Q[j])
                    if j == 1:
                        sir_values[j] = [np.array([arr.squeeze()]) for arr in sir_values[j]]
                        obj1_optimal_values[j] = [np.array([arr.squeeze()]) for arr in obj1_optimal_values[j]]
                        obj2_optimal_values[j] = [np.array([arr.squeeze()]) for arr in obj2_optimal_values[j]]
                    if j == 2:
                        sir_values[j] = [np.array([arr.squeeze()]) for arr in sir_values[j]]
                        obj1_optimal_values[j] = [np.array([arr.squeeze()]) for arr in obj1_optimal_values[j]]
                        obj2_optimal_values[j] = [np.array([arr.squeeze()]) for arr in obj2_optimal_values[j]]
                        
            plt.figure(figsize=(12, 8))
            # Subplot 1 (1 row, 2 columns, 1st plot)
            plt.subplot(2, 3, 1)
            plotyy(np.arange(0.1, 1.0, 0.1), sir_values[0], obj1_optimal_values[0], obj2_optimal_values[0],'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal Absolute error')
            # Subplot 2 (1 row, 2 columns, 2nd plot)
            plt.subplot(2, 3, 2)
            plotyy(np.arange(0.1, 1.0, 0.1), sir_values[1], obj1_optimal_values[1], obj2_optimal_values[1], 'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal MSE')
            plt.subplot(2, 3, 3)
            plotyy(np.arange(0.1, 1.0, 0.1), sir_values[2], obj1_optimal_values[2], obj2_optimal_values[2], 'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal Risk')
            plt.subplot(2, 3, 4)
            plotyy(np.arange(0.1, 1.0, 0.1), [0 for _ in range(9)], obj1_optimal_Q[0], obj2_optimal_Q[0],'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal lag Q for Absolute error')
            plt.subplot(2, 3, 5)
            plotyy(np.arange(0.1, 1.0, 0.1), [0 for _ in range(9)], obj1_optimal_Q[1], obj2_optimal_Q[1], 'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal lag Q for MSE')                
            plt.subplot(2, 3, 6)
            plotyy(np.arange(0.1, 1.0, 0.1), [0 for _ in range(9)], obj1_optimal_Q[2], obj2_optimal_Q[2], 'SIR', 'Model 1','Model 2' ,'' , 'from 0.1 to 0.9, step size 0.1', 'Optimal lag Q for Risk')
            plt.savefig(fr'C:\Users\yhuang73\Desktop\figuretestAR2_{n_obs}_{sigma}_{n_rep}_set2\arcoeff{"_".join(map(str, [ round(val, 1) for val in [j,i,k,k] ]))}.png', format='png')
            plt.close()

In [93]:
import os
import shutil
def Removefiles(folder_path):
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Removes directory and contents            
a1 = 2; a2 = 3; initial_value = [[10,10],[10,10],[10,10],[10,10]]; n_burn = 1000; n_obs = 100; H = 5; S = 10; sigma = 1; n_rep = 50
# change to number of samples = 100? Insufficient data?
folder_path1 = fr'C:\Users\yhuang73\Desktop\figuretestAR2_{n_obs}_{sigma}_{n_rep}'
folder_path2 = fr'C:\Users\yhuang73\Desktop\datatestAR2_{n_obs}_{sigma}_{n_rep}'
# Removefiles(folder_path2)
arr = []
for i in np.arange(0.1, 1.0, 0.1):
    for j in np.arange(0.1, 1.0, 0.1):
        arr.append([[i, j, 0.2, 0.2], [0.1, 0.1, 0.1, 0.1]])       
        arr.append([[i, j, 0.4, 0.4], [0.1, 0.1, 0.1, 0.1]])        
        arr.append([[i, j, 0.6, 0.6], [0.1, 0.1, 0.1, 0.1]])
for index in range(len(arr)):
    arr[index] = [[round(num, 2) for num in sublist] for sublist in arr[index]]
for ar_coeff in arr:
    Generation(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, sigma, n_rep)

KeyboardInterrupt: 

In [50]:
Removefiles(folder_path1)

In [ ]:
a1 = 2; a2 = 3; initial_value = [[1,1],[1,1],[1,1],[1,1]];
n_burn = 1000; n_obs = 100; H = 5; S = 10; sigma = 1; n_rep = 50
Graph(H, S, n_rep)

C:\Users\yhuang73\AppData\Local\Temp\ipykernel_18868\2439906099.py:225: RuntimeWarning: invalid value encountered in scalar divide
  error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
C:\Users\yhuang73\AppData\Local\Temp\ipykernel_18868\2439906099.py:225: RuntimeWarning: invalid value encountered in scalar divide
  error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
C:\Users\yhuang73\AppData\Local\Temp\ipykernel_18868\2439906099.py:225: RuntimeWarning: invalid value encountered in scalar divide
  error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
C:\Users\yhuang73\AppData\Local\Temp\ipykernel_18868\2439906099.py:225: RuntimeWarning: invalid value encountered in scalar divide
  error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
C:\Users\yhuang73\AppData\Local\Temp\ipykernel_18868\2439906099.py:225: RuntimeWarning: invalid value encountered in scalar divide
  error_1[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
